In [96]:
import pandas as pd

In [97]:
genius = pd.read_csv('/content/genius.csv')
hitmotop = pd.read_csv('/content/hitmotop_artist_tracks.csv')
spotify = pd.read_csv('/content/spotify_artists.csv')
artists = pd.read_csv('/content/artists_stats.csv')

In [98]:
def clean_text_data(df):
    df_clean = df.copy()
    string_columns = df_clean.select_dtypes(include=['object']).columns
    for col in string_columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()

    return df_clean

In [99]:
genius = genius.rename(columns={
    'title': 'song_title'
})

genius.columns = genius.columns.str.lower()
genius = clean_text_data(genius)

print(f"кол-во дубликатов до очистки: {genius.duplicated().sum()}")
genius = genius.drop_duplicates()
print(f"кол-во дубликатов после очистки: {genius.duplicated().sum()}")

columns_to_keep = ['artist', 'song_title', 'page_views']
genius = genius[columns_to_keep]

genius.head()

кол-во дубликатов до очистки: 0
кол-во дубликатов после очистки: 0


,artist,song_title,page_views
0,Drake,God’s Plan,9933595
1,Drake,In My Feelings,7853393
2,Drake,Hotline Bling,7264399
3,Drake,One Dance,6288104
4,Drake,"Hold On, We’re Going Home",5655239


In [101]:
hitmotop = hitmotop.rename(columns={
    'Исполнитель': 'artist',
    'Название песни': 'song_title',
    'продолжительность': 'song_long'
})

hitmotop.columns = hitmotop.columns.str.lower()
hitmotop = clean_text_data(hitmotop)

print(f"кол-во дубликатов до очистки: {hitmotop.duplicated().sum()}")
hitmotop = hitmotop.drop_duplicates()
print(f"кол-во дубликатов после очистки: {hitmotop.duplicated().sum()}")

columns_to_keep = ['artist', 'song_title', 'song_long']
hitmotop = hitmotop[columns_to_keep]

hitmotop.head()

кол-во дубликатов до очистки: 0
кол-во дубликатов после очистки: 0


,artist,song_title,song_long
0,Drake,Forever\n (Album Version (Expli...,05:57
1,Drake,I'm On One\n (Explicit Version),04:58
2,Drake,Best I Ever Had,04:19
3,Drake,God's Plan,03:19
4,Drake,Love Me\n (Album Version (Expli...,04:15


In [102]:
spotify_columns = spotify.columns.tolist()
spotify_columns[0], spotify_columns[1] = spotify_columns[1], spotify_columns[0]
spotify.columns = spotify_columns

spotify = spotify.rename(columns={
    'Artist': 'artist'
})

spotify.columns = spotify.columns.str.lower()
spotify = clean_text_data(spotify)

print(f"кол-во дубликатов до очистки: {spotify.duplicated().sum()}")
spotify = spotify.drop_duplicates()
print(f"кол-во дубликатов после очистки: {spotify.duplicated().sum()}")

spotify.head()

кол-во дубликатов до очистки: 0
кол-во дубликатов после очистки: 0


,artist,rank,daily streams,peak daily,monthly listeners
0,Drake,"122,491.8",48.224,"83,627.9","46,307.8"
1,Taylor Swift,"114,464.0",73.087,"110,924.1","101,320.6"
2,Bad Bunny,"108,856.3",48.179,"69,861.9","40,392.9"
3,The Weeknd,"86,408.1",40.980,"69,564.2","46,496.7"
4,Justin Bieber,"65,506.9",26.747,"40,668.3","24,976.2"


In [103]:
artists.columns = artists.columns.str.lower()
artists = clean_text_data(artists)

print(f"кол-во дубликатов до очистки: {artists.duplicated().sum()}")
artists = artists.drop_duplicates()
print(f"кол-во дубликатов после очистки: {artists.duplicated().sum()}")

artists = artists.drop(columns=['artist_id'])

artists.head()

кол-во дубликатов до очистки: 0
кол-во дубликатов после очистки: 0


,artist,followers,genres,popularity,type
0,Drake,103393441.0,rap,95.0,artist
1,Taylor Swift,145960477.0,nan,100.0,artist
2,Bad Bunny,102944627.0,"reggaeton, trap latino, urbano latino, latin",96.0,artist
3,The Weeknd,113413144.0,nan,94.0,artist
4,Justin Bieber,85170938.0,nan,91.0,artist


In [104]:
merged_genius_hitmotop = genius.merge(
    hitmotop,
    on=['artist', 'song_title'],
    how='left',
    suffixes=('_genius', '_hitmotop')
)


merged_genius_hitmotop = merged_genius_hitmotop.drop_duplicates()


print(f"Размер объединенного датафрейма: {merged_genius_hitmotop.shape}")
merged_genius_hitmotop.head()


Размер объединенного датафрейма: (42666, 4)


,artist,song_title,page_views,song_long
0,Drake,God’s Plan,9933595,NaN
1,Drake,In My Feelings,7853393,03:38
2,Drake,Hotline Bling,7264399,04:27
3,Drake,One Dance,6288104,02:54
5,Drake,"Hold On, We’re Going Home",5655239,NaN


In [105]:
merged_genius_hitmotop.to_csv('merged_genius_hitmotop.csv', index=False, encoding='utf-8')

In [106]:
all_artists = pd.concat([
    merged_genius_hitmotop[['artist']],
    spotify[['artist']],
    artists[['artist']]
]).drop_duplicates().reset_index(drop=True)

print(f"Всего уникальных артистов: {len(all_artists)}")


Всего уникальных артистов: 6007


In [107]:
final_merged_artists = all_artists.merge(artists, on='artist', how='left')
print("После объединения с artists:")
print(f"Размер: {final_merged_artists.shape}")

final_merged_artists = final_merged_artists.merge(spotify, on='artist', how='left')
print("После объединения с spotify:")
print(f"Размер: {final_merged_artists.shape}")

После объединения с artists:
Размер: (6007, 5)
После объединения с spotify:
Размер: (6009, 9)


In [108]:
genius_hitmotop_stats = merged_genius_hitmotop.groupby('artist').agg({
    'page_views': 'sum',
    'song_title': 'count',
    'song_long': 'count'
}).rename(columns={
    'song_title': 'total_songs_count',
    'song_long': 'songs_with_duration_count',
    'page_views': 'total_page_views'
}).reset_index()

final_merged_artists = final_merged_artists.merge(genius_hitmotop_stats, on='artist', how='left')

print("После добавления статистики из merged_genius_hitmotop:")
print(f"Размер: {final_merged_artists.shape}")

final_merged_artists = final_merged_artists.dropna()

final_merged_artists.head()

После добавления статистики из merged_genius_hitmotop:
Размер: (6009, 12)


,artist,followers,genres,popularity,type,rank,daily streams,peak daily,monthly listeners,total_page_views,total_songs_count,songs_with_duration_count
0,Drake,103393441.0,rap,95.0,artist,"122,491.8",48.224,"83,627.9","46,307.8",127456306.0,47.0,14.0
1,Taylor Swift,145960477.0,nan,100.0,artist,"114,464.0",73.087,"110,924.1","101,320.6",91931989.0,41.0,12.0
2,Cardi B,25984396.0,nan,81.0,artist,"21,077.3",6.050,"9,923.9","4,005.8",46310865.0,18.0,0.0
5,Bad Bunny,102944627.0,"reggaeton, trap latino, urbano latino, latin",96.0,artist,"108,856.3",48.179,"69,861.9","40,392.9",19689780.0,51.0,8.0
6,JHAYCO,9424183.0,"reggaeton, trap latino, urbano latino",80.0,artist,"15,882.5",5.066,"6,822.5","1,881.5",2446826.0,11.0,0.0


In [109]:
final_merged_artists.to_csv('final_merged_artists.csv', index=False, encoding='utf-8')